# HMM PoS tagger

## Introduction

## HMM implementation

In [ ]:
# Implementation of the HMM model

## Viterbi algorithm

In [ ]:
# Implementation of the viterbi algorithm

##Experiments

### In-domain

### Out-of-domain

## Conclusion